CRAWLING

html von abstimmungen crawlen um url für xlsx zu bekommen

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from urllib.parse import urljoin
import re
import pandas as pd
import uuid
import time 

# URL of the website
url = "https://www.bundestag.de/parlament/plenum/abstimmung/liste"
base_url = "https://www.bundestag.de"
# Set up the Selenium WebDriver for Chrome
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
service = Service('/usr/bin/chromedriver')
driver = webdriver.Chrome(service=service, options=options)
    
# Open the webpage
driver.get(url)
    
page_number = 1
while True:
    # Wait for the specific element that indicates the content has fully loaded
    time.sleep(10)
        
    # Save the page source to a file for inspection
    with open(f"dat/response_page.html", "w", encoding='utf-8') as file:
        file.write(driver.page_source)
        
    print(f"HTML content saved to response_page_{page_number}.html")
        
    # Find the "Next" button
    try:
        next_button = driver.find_element(By.CLASS_NAME, 'slick-next.slick-arrow')
        if next_button.is_enabled():
            next_button.click()
            page_number += 1
            # Wait for the new content to load
            time.sleep(5)  # Adjust the sleep time as needed
        else:
            print("No more pages to navigate.")
            break
    except Exception as e:
        print(f"An error occurred: {e}")
        break
    


ModuleNotFoundError: No module named 'selenium'

Abstimmungen xlsx url, datum und thema aus .html extrahieren

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import uuid

# Read the locally saved HTML file
with open("dat/response_page.html", "r", encoding='utf-8') as file:
    page_content = file.read()

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(page_content, 'html.parser')

# Initialize an empty DataFrame
df = pd.DataFrame(columns=['Date', 'Topic', 'XLSX_URL', 'PDF_URL', 'FILENAME'])

# Find all <td> elements with data-th="Dokument"
td_elements = soup.find_all('td', {'data-th': 'Dokument'})

# Loop over all the <td> elements
for td in td_elements:
    try:
        # Find the <div> with the class 'bt-documents-description'
        div = td.find('div', class_='bt-documents-description')
        
        # Extract the date and topic from the <p> tag
        p_tag = div.find('p')
        date_topic = p_tag.text.strip().split(':', 1)  # Split by the first occurrence of ':'
        
        # Ensure the split operation produced the expected number of elements
        if len(date_topic) == 2:
            date = date_topic[0].strip()
            topic = date_topic[1].strip()
        else:
            print(f"Unexpected format in <p> tag: {p_tag.text.strip()}")
            continue
        
        # Initialize URLs
        xlsx_url = None
        pdf_url = None
        
        # Find all <a> tags within the <ul> with the class 'bt-linkliste'
        a_tags = div.find('ul', class_='bt-linkliste').find_all('a')
        
        for a_tag in a_tags:
            href = a_tag['href']
            title = a_tag['title']
            if 'XLSX' in title:
                xlsx_url = href
            elif 'PDF' in title:
                pdf_url = href
        
        # Generate a unique filename using UUID
        filename = str(uuid.uuid4())
        
        # Create a new DataFrame row
        new_row = pd.DataFrame({'Date': [date], 'Topic': [topic], 'XLSX_URL': [xlsx_url], 'PDF_URL': [pdf_url], 'FILENAME': [filename]})
        
        # Concatenate the new row to the existing DataFrame
        df = pd.concat([df, new_row], ignore_index=True)
    
    except Exception as e:
        print(f"An error occurred while processing an element: {e}")

# Display the DataFrame
print(df)

# Save the DataFrame to a CSV file
df.to_csv('dat/dataframe_with_links.csv', index=False)

Unexpected format in <p> tag: Änderung des Schwangerschaftskonfliktgesetzes
Unexpected format in <p> tag: Wachstumschancengesetz; KfZ_Haftpflichtversicherung (BeschlEmpf Vermittlungsausschuss)
           Date                                              Topic  \
0    18.10.2024  Ablehnung eines Antrags zu Drittstaatenangehör...   
1    18.10.2024         Ablehnung eines Antrags zu Messerangriffen   
2    18.10.2024  Dritter Entschließungsantrag zur „Verbesserung...   
3    18.10.2024  Zweiter Entschließungsantrag zur „Verbesserung...   
4    18.10.2024  Erster Entschließungsantrag zur „Verbesserung ...   
..          ...                                                ...   
293  13.11.2019  Gesetzentwurf Direktzahlungen-Durchführungsgesetz   
294  07.11.2019                 Änderungsantrag der FDP (19/14883)   
295  07.11.2019                 Änderungsantrag der FDP (19/14882)   
296  24.10.2019                   Wettbewerbsfähige Landwirtschaft   
297  24.10.2019                      

Xlsx-abstimmung crawlen 

In [ ]:
import pandas as pd
import requests
import os
import time
from urllib.parse import urljoin

# Base URL
base_url = "https://www.bundestag.de"

# Read the DataFrame from the CSV file
df = pd.read_csv('dat/dataframe_with_links.csv')

# Create the 'dat' folder if it doesn't exist
if not os.path.exists('index'):
    os.makedirs('index')

# Function to download and save the XLSX file
def download_xlsx(url, filename):
    try:
        # Send a GET request to download the file
        response = requests.get(url)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Save the file locally in the 'dat' folder
            with open(f"index/{filename}.xlsx", "wb") as file:
                file.write(response.content)
            print(f"File {filename}.xlsx downloaded successfully.")
        else:
            print(f"Failed to download file {filename}. Status code: {response.status_code}")
    except Exception as e:
        print(f"An error occurred while downloading {filename}: {e}")

# Loop over each row in the DataFrame
for index, row in df.iterrows():
    xlsx_url = row['XLSX_URL']
    filename = row['FILENAME']
    
    # Append the base URL to the relative URL
    full_url = urljoin(base_url, xlsx_url)
    
    # Download and save the XLSX file
    download_xlsx(full_url, filename)
    
    # Add a delay to avoid getting banned
    time.sleep(2)  # Adjust the sleep time as needed

File 684f6618-e1ec-4ae1-8af6-e21dc6c86046.xlsx downloaded successfully.
File 9d58672d-d74e-431e-a205-5c55d5201e65.xlsx downloaded successfully.
File 55721ae3-9e42-4f20-972f-e3987d7c248b.xlsx downloaded successfully.
File 73baa924-a7af-4249-9c16-f6bb71ebf849.xlsx downloaded successfully.
File c117bd57-d16e-48cc-a017-8b94bbc3713c.xlsx downloaded successfully.
File 12da23f5-f08c-4b13-ad20-3b5b9bc6ead1.xlsx downloaded successfully.
File 78bc68a1-7d1d-4626-9b71-a171f119bcc4.xlsx downloaded successfully.
File 65a313b8-0692-42cb-8802-a2af5de6825e.xlsx downloaded successfully.
File 53e87bb3-3d4f-4023-a2bd-17c347870fc1.xlsx downloaded successfully.
File 5174016f-7d08-4b60-9314-66b51d0534ac.xlsx downloaded successfully.
File b57f8d33-82b5-4182-a01f-05a739dd0d27.xlsx downloaded successfully.
File e0bca1e4-b4ff-42d5-98fc-ba548cbabda3.xlsx downloaded successfully.
File eb865566-5478-451c-8012-ca3a898a45c2.xlsx downloaded successfully.
File 9953dc12-de7b-47fd-8dea-8990c86ef15c.xlsx downloaded succes

abstimmung einlesen und mit thema und datum verbinden zu einem dataframe

todo: zweiter dataframe mit id, datum, thema und dann über inner join

In [50]:
import pandas as pd
import os

# Read the DataFrame from the CSV file
df_links = pd.read_csv('dat/dataframe_with_links.csv')

# Initialize an empty DataFrame to store the combined data
combined_df = pd.DataFrame()

# Loop over each row in the DataFrame
for index, row in df_links.iterrows():
    filename = row['FILENAME']
    
    # Construct the file path
    file_path = f"index/{filename}.xlsx"
    
    # Check if the file exists
    if os.path.exists(file_path):
        # Load the XLSX data into a DataFrame
        xlsx_df = pd.read_excel(file_path)
        
        # Add the Date and Topic columns to the DataFrame
        xlsx_df['FILENAME'] = filename
        
        # Append the data to the combined DataFrame
        combined_df = pd.concat([combined_df, xlsx_df], ignore_index=True)
    else:
        print(f"File {file_path} does not exist.")

# Display the combined DataFrame
print(combined_df)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv('dat/combined_data.csv', index=False)

        Wahlperiode  Sitzungnr  Abstimmnr Fraktion/Gruppe             Name  \
0                20        195          9             SPD             Abdi   
1                20        195          9             SPD        Ahmetovic   
2                20        195          9             SPD  Alabali-Radovan   
3                20        195          9             SPD           Andres   
4                20        195          9             SPD            Annen   
...             ...        ...        ...             ...              ...   
215811           19        121          2         BÜ90/GR     Zickenheiner   
215812           19        121          2    Fraktionslos            Bülow   
215813           19        121          2    Fraktionslos           Kamann   
215814           19        121          2    Fraktionslos          Mieruch   
215815           19        121          2    Fraktionslos            Petry   

        Vorname Titel  ja  nein  Enthaltung  ungültig  nichtabg

Liste der Abgeordenten nach Wahlperiode:
https://search.dip.bundestag.de/api/v1/swagger-ui/#/Personenstammdaten/getPersonList

Liste der Drucksachen nach id (aus pdf):
https://search.dip.bundestag.de/api/v1/swagger-ui/#/Drucksachen/getDrucksacheTextList

In [60]:
# Filter the DataFrame to include only votes after 2020
filtered_df = combined_df[combined_df['Wahlperiode'] == 20]

combined_df.head()


,Wahlperiode,Sitzungnr,Abstimmnr,Fraktion/Gruppe,Name,Vorname,Titel,ja,nein,Enthaltung,ungültig,nichtabgegeben,Bezeichnung,Bemerkung,FILENAME,AbgNr
0,20,195,9,SPD,Abdi,Sanae,NaN,0,0,0,0,1,Sanae Abdi,NaN,684f6618-e1ec-4ae1-8af6-e21dc6c86046,NaN
1,20,195,9,SPD,Ahmetovic,Adis,NaN,0,0,0,0,1,Adis Ahmetovic,NaN,684f6618-e1ec-4ae1-8af6-e21dc6c86046,NaN
2,20,195,9,SPD,Alabali-Radovan,Reem,NaN,1,0,0,0,0,Reem Alabali-Radovan,NaN,684f6618-e1ec-4ae1-8af6-e21dc6c86046,NaN
3,20,195,9,SPD,Andres,Dagmar,NaN,1,0,0,0,0,Dagmar Andres,NaN,684f6618-e1ec-4ae1-8af6-e21dc6c86046,NaN
4,20,195,9,SPD,Annen,Niels,NaN,1,0,0,0,0,Niels Annen,NaN,684f6618-e1ec-4ae1-8af6-e21dc6c86046,NaN


add topic category to vote

In [ ]:
# get all unique Topics
unique_topics = df_links["Topic"]
print(unique_topics)

# extrahierte topics: 
categories = "Gesundheit, Einwanderung, Sicherheit, Energie, Wirtschaft, Rechtsreform, Außenpolitik, Regierungspolitik, Umwelt"

0      Ablehnung eines Antrags zu Drittstaatenangehör...
1             Ablehnung eines Antrags zu Messerangriffen
2      Dritter Entschließungsantrag zur „Verbesserung...
3      Zweiter Entschließungsantrag zur „Verbesserung...
4      Erster Entschließungsantrag zur „Verbesserung ...
                             ...                        
293    Gesetzentwurf Direktzahlungen-Durchführungsgesetz
294                   Änderungsantrag der FDP (19/14883)
295                   Änderungsantrag der FDP (19/14882)
296                     Wettbewerbsfähige Landwirtschaft
297                                      Russlandpolitik
Name: Topic, Length: 298, dtype: object


In [ ]:
import os
import time
from groq import Groq

client = Groq(
    api_key="xxx",
)
votes_with_categories = df_links.copy()

for categorie in categories.split(", "):
    votes_with_categories[categorie] = 0

for i, row in votes_with_categories.iterrows():
    if i == 300:
        votes_with_categories.head()

        votes_with_categories = pd.merge(combined_df, votes_with_categories , on='FILENAME')


        votes_with_categories.to_csv("dat/votes_with_categories.csv")
        break
    else: 
        
        sentence = row["Topic"]
        content = f'Ordne den folgenden Satz dem passenden Thema zu: "{sentence}". Wähle aus den Themen: {categories}. Gib maximal zwei relevante Themen mit Leerzeichen getrennt zurück, ohne zusätzlichen Text.'
        
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": content,
                }
            ],
            model="llama3-8b-8192",
        )
        answer = chat_completion.choices[0].message.content
        for topic in answer.split(" "):
            if topic in categories:
                votes_with_categories.at[i, topic] = 1
            else:
                print(f'this topic has no valid answer: {sentence}. The answer is: {answer}')
        time.sleep(1)



votes_with_categories = pd.merge(combined_df, votes_with_categories , on='FILENAME')
votes_with_categories.to_csv("dat/votes_with_categories.csv")

this topic has no valid answer: Wahl zum Europäischen Parlament. The answer is: Wahl zum Europäischen Parlament: Außenpolitik Wirtschaft
this topic has no valid answer: Wahl zum Europäischen Parlament. The answer is: Wahl zum Europäischen Parlament: Außenpolitik Wirtschaft
this topic has no valid answer: Wahl zum Europäischen Parlament. The answer is: Wahl zum Europäischen Parlament: Außenpolitik Wirtschaft
this topic has no valid answer: Wahl zum Europäischen Parlament. The answer is: Wahl zum Europäischen Parlament: Außenpolitik Wirtschaft
this topic has no valid answer: Mindestlohn. The answer is: Wirtschaft Erechtsreform
this topic has no valid answer: Verwendung geschlechtergerechter Sprache. The answer is: Rechtsreform Verwendung
this topic has no valid answer: Klimaschutz. The answer is: Energie Klimaschutz
this topic has no valid answer: Lieferketten. The answer is: Wirtschaft Lieferketten
this topic has no valid answer: Haushaltsjahr 2020. The answer is: Wirtschaft Haushalts
t